In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
chunk_size = 200
search_ef = 0 # 0 for default, recommend 100

if search_ef == 0:
    collection_name = f"mycollection_{chunk_size}"
else:
    collection_name = f"mycollection_{chunk_size}_ef_{search_ef}"

print(f"検索対象のコレクション名: {collection_name}")

model = "gpt-4o"
embedding_model = "text-embedding-3-large"
temperature = 0

top_k = 10

# 検索タイプ - similarity
#search_type="similarity"

# 検索タイプ - similarity_score_threshold
#search_type="similarity_score_threshold"
score_threshold = 0.3

# 検索タイプ - mmr
search_type="mmr" # for mmr
fetch_k = top_k * 2 # for mmr : Amount of documents to pass to MMR algorithm
lambda_mult = 1.0 # for mmr : 1 for minimum diversity and 0 for maximum. (Default: 0.5)


検索対象のコレクション名: mycollection_200


### 埋め込みモデルの準備

In [3]:
from langchain_openai import OpenAIEmbeddings

# 埋め込みモデルの準備
embeddings = OpenAIEmbeddings(model=embedding_model)

### VectorStoreの準備

In [4]:
import chromadb
from langchain_chroma import Chroma


# Chromaクライアントを初期化
try:
    persistent_client = chromadb.PersistentClient(path="./chroma_db")

    collections = persistent_client.list_collections()
    print("現在のコレクション一覧:")
    for collection in collections:
        print(f"- {collection.name}")

    vector_store = Chroma(
        client=persistent_client,
        collection_name=collection_name,
        embedding_function=embeddings,
        collection_metadata={"hnsw:space": "ip"} 
    )
    print(f"コレクション {collection_name} のインスタンスを作成しました")

except Exception as e:
    print(f"Chromaデータベースの操作中にエラーが発生しました: {str(e)}")


現在のコレクション一覧:
- mycollection_100
- mycollection_200_ef_100
- mycollection_500_ef_100
- mycollection_1000
- mycollection_200
- mycollection_500
コレクション mycollection_200 のインスタンスを作成しました


In [5]:
# テスト用の質問
query="高町なのはが小学生のときに通っていた学校は？"
# 質問応答
result = vector_store.similarity_search(query, k=top_k)
result

[Document(metadata={'source': 'data\\Wikipedia-マギアレコード_魔法少女まどか☆マギカ外伝.pdf'}, page_content='神浜市の区新⻄区（しんせいく）ももこ‧かえで‧レナのチームが活動する地区で、やちよの⾃宅である「みかづき荘」や、みたまの「調整屋[注 16]」、うい‧灯花‧ねむが⼊院していた「⾥⾒メディカルセンター」などがある。地区内にある学校は神浜市⽴⼤附属学校で、いろはやももこたちが通っている。⽔名区（みずなく）かつての城下町で、歴史を感じさせる古い街並みが残る地区[物語 3]。'),
 Document(metadata={'source': 'data\\Wikipedia-マギアレコード_魔法少女まどか☆マギカ外伝.pdf'}, page_content='いろはが以前所属していた宝崎市⽴第⼀中学校や、那由他が所属する宝崎順⼼学園、燦とミユリが所属する宝崎市光塚中等教育学校、いろは‧ういや那由他がそれぞれかつて暮らしていた家もある。市内の光塚地域では、伝統の⽕祭りが⾏われている[物語 17]。⾒滝原市（みたきはらし）原作アニメ『魔法少⼥まどか☆マギカ』の舞台となる町。'),
 Document(metadata={'source': 'data\\Wikipedia-マギアレコード_魔法少女まどか☆マギカ外伝.pdf'}, page_content='また、ひめなが所属する「神浜未来アカデミー」もある[物語 8]。⼯匠区（こうしょうく）「男⼦は職⼈として尊重されながら仕事をし、⼥は男を⽀えながら家を守る」という、やや時代錯誤な⾵習が残っている地域[物語 9]。プロミストブラッドが神浜での拠点とする旧⾞両基地がある。\x0c地区内にある学校は⼯匠学舎で、⽉咲が通っている。'),
 Document(metadata={'source': 'data\\Wikipedia-高町なのは.pdf'}, page_content='⾼町なのは出典: フリー百科事典『ウィキペディア（Wikipedia）』魔法少⼥リリカルなのはシリーズ > 登場⼈物 > ⾼町なのは⾼町 なのは（たかまち なのは）は、テレビアニメ『魔法少⼥リリカルなのは』シリーズに登場する架空の⼈物で、本編の主⼈公。『魔法戦記リリカルなのはF

In [8]:
# 質問応答の結果を見やすくフォーマットする関数
def format_search_response(result):
    print("\n検索結果:")
    for i, doc in enumerate(result, 1):
        print(f"\n--- ドキュメント {i} ---")
        print(f"ソース: {doc.metadata['source']}")
        print(f"文字数: {len(doc.page_content)}")
        print(f"内容: {doc.page_content[:1000]}...")  # 長すぎる場合は省略

format_search_response(result)


検索結果:

--- ドキュメント 1 ---
ソース: data\Wikipedia-マギアレコード_魔法少女まどか☆マギカ外伝.pdf
文字数: 158
内容: いろはが以前所属していた宝崎市⽴第⼀中学校や、那由他が所属する宝崎順⼼学園、燦とミユリが所属する宝崎市光塚中等教育学校、いろは‧ういや那由他がそれぞれかつて暮らしていた家もある。市内の光塚地域では、伝統の⽕祭りが⾏われている[物語 17]。⾒滝原市（みたきはらし）原作アニメ『魔法少⼥まどか☆マギカ』の舞台となる町。...

--- ドキュメント 2 ---
ソース: data\Wikipedia-マギアレコード_魔法少女まどか☆マギカ外伝.pdf
文字数: 185
内容: 神浜市の区新⻄区（しんせいく）ももこ‧かえで‧レナのチームが活動する地区で、やちよの⾃宅である「みかづき荘」や、みたまの「調整屋[注 16]」、うい‧灯花‧ねむが⼊院していた「⾥⾒メディカルセンター」などがある。地区内にある学校は神浜市⽴⼤附属学校で、いろはやももこたちが通っている。⽔名区（みずなく）かつての城下町で、歴史を感じさせる古い街並みが残る地区[物語 3]。...

--- ドキュメント 3 ---
ソース: data\Wikipedia-マギアレコード_魔法少女まどか☆マギカ外伝.pdf
文字数: 159
内容: また、ひめなが所属する「神浜未来アカデミー」もある[物語 8]。⼯匠区（こうしょうく）「男⼦は職⼈として尊重されながら仕事をし、⼥は男を⽀えながら家を守る」という、やや時代錯誤な⾵習が残っている地域[物語 9]。プロミストブラッドが神浜での拠点とする旧⾞両基地がある。地区内にある学校は⼯匠学舎で、⽉咲が通っている。...

--- ドキュメント 4 ---
ソース: data\Wikipedia-高町なのは.pdf
文字数: 197
内容: ⾼町なのは出典: フリー百科事典『ウィキペディア（Wikipedia）』魔法少⼥リリカルなのはシリーズ > 登場⼈物 > ⾼町なのは⾼町 なのは（たかまち なのは）は、テレビアニメ『魔法少⼥リリカルなのは』シリーズに登場する架空の⼈物で、本編の主⼈公。『魔法戦記リリカルなのはForce』および『魔法少⼥リリカルなのはViVid』では年⻑者として新しい主⼈公たちを⾒守る⽴ち

In [9]:
vector_store.similarity_search_with_score(query, k=top_k)

[(Document(metadata={'source': 'data\\Wikipedia-マギアレコード_魔法少女まどか☆マギカ外伝.pdf'}, page_content='いろはが以前所属していた宝崎市⽴第⼀中学校や、那由他が所属する宝崎順⼼学園、燦とミユリが所属する宝崎市光塚中等教育学校、いろは‧ういや那由他がそれぞれかつて暮らしていた家もある。市内の光塚地域では、伝統の⽕祭りが⾏われている[物語 17]。⾒滝原市（みたきはらし）原作アニメ『魔法少⼥まどか☆マギカ』の舞台となる町。'),
  0.5942181348800659),
 (Document(metadata={'source': 'data\\Wikipedia-マギアレコード_魔法少女まどか☆マギカ外伝.pdf'}, page_content='神浜市の区新⻄区（しんせいく）ももこ‧かえで‧レナのチームが活動する地区で、やちよの⾃宅である「みかづき荘」や、みたまの「調整屋[注 16]」、うい‧灯花‧ねむが⼊院していた「⾥⾒メディカルセンター」などがある。地区内にある学校は神浜市⽴⼤附属学校で、いろはやももこたちが通っている。⽔名区（みずなく）かつての城下町で、歴史を感じさせる古い街並みが残る地区[物語 3]。'),
  0.5961878299713135),
 (Document(metadata={'source': 'data\\Wikipedia-マギアレコード_魔法少女まどか☆マギカ外伝.pdf'}, page_content='また、ひめなが所属する「神浜未来アカデミー」もある[物語 8]。⼯匠区（こうしょうく）「男⼦は職⼈として尊重されながら仕事をし、⼥は男を⽀えながら家を守る」という、やや時代錯誤な⾵習が残っている地域[物語 9]。プロミストブラッドが神浜での拠点とする旧⾞両基地がある。\x0c地区内にある学校は⼯匠学舎で、⽉咲が通っている。'),
  0.5990170240402222),
 (Document(metadata={'source': 'data\\Wikipedia-高町なのは.pdf'}, page_content='⾼町なのは出典: フリー百科事典『ウィキペディア（Wikipedia）』魔法少⼥リリカルなのはシリーズ > 登場⼈物 > ⾼町なのは

In [130]:
vector_store.similarity_search_with_score(query, k=top_k, filter={"source": "data\\Wikipedia-鹿目まどか.pdf"})


[(Document(metadata={'source': 'data\\Wikipedia-鹿目まどか.pdf'}, page_content='こうして「円環の理」としてのまどかは、さやかとなぎさの助⼒を得て、インキュベーターの企みを阻⽌することに成功する。この時まどかは実戦において、左右どちらの射型でも⼸を射る事ができる描写がある。その後、「円環の理」としての記憶と⼒が戻ったまどかが、ほむらを導こうと天から訪れる。だがこの時、ほむらが「円環の理」の⼒の⼀部をもぎ取り、因果律を再構成させるという予想外の⾏動を起こす。'),
  0.5185303233618257),
 (Document(metadata={'source': 'data\\Wikipedia-鹿目まどか.pdf'}, page_content='『マギアレコード 魔法少⼥まどか☆マギカ外伝』の舞台版では丹⽣明⾥（けやき坂46〈現‧⽇向坂46〉）が演じる[4]。役柄⾒滝原中学校2年⽣。家族構成は⺟の詢⼦、⽗の知久、弟のタツヤの4⼈で、同じクラスの美樹さやかと志筑仁美とは友⼈関係にある。本編時間軸では魔法少⼥としては途⽅もない素質を持っており、執拗にキュゥべえから契約を迫られることになる。魔⼥化した際には救済の魔⼥に変化する。'),
  0.5480213168385603),
 (Document(metadata={'source': 'data\\Wikipedia-鹿目まどか.pdf'}, page_content='性別誕⽣⽇年齢出⾝地学年⾝⻑⾎液型武器ー‧カバノス（英語版）⼥10⽉3⽇[注 1]14歳[2]⽇本中学2年⽣152cm[3]A型⼸キャラクターデザイン 蒼樹うめ（原案）、岸⽥出典隆宏アニプレックス発⾏フリーペーパー「ANI-©OM」2010年12⽉号「ヒロイン：⿅⽬まどかへの100の質問」ほむらは「ほむらちゃん」、さやかは「さやかちゃん」、マミは「マミさん」、杏⼦は「杏⼦ちゃん」と呼ぶ。'),
  0.549561697427568),
 (Document(metadata={'source': 'data\\Wikipedia-鹿目まどか.pdf'}, page_content='⿅⽬まどか出典: フリー百科事典『ウィキペディア（Wikipedia）』魔法少⼥まどか☆

In [10]:
vector_store.max_marginal_relevance_search(query, k=top_k, fetch_k=fetch_k, lambda_mult=lambda_mult)

[Document(metadata={'source': 'data\\Wikipedia-マギアレコード_魔法少女まどか☆マギカ外伝.pdf'}, page_content='いろはが以前所属していた宝崎市⽴第⼀中学校や、那由他が所属する宝崎順⼼学園、燦とミユリが所属する宝崎市光塚中等教育学校、いろは‧ういや那由他がそれぞれかつて暮らしていた家もある。市内の光塚地域では、伝統の⽕祭りが⾏われている[物語 17]。⾒滝原市（みたきはらし）原作アニメ『魔法少⼥まどか☆マギカ』の舞台となる町。'),
 Document(metadata={'source': 'data\\Wikipedia-マギアレコード_魔法少女まどか☆マギカ外伝.pdf'}, page_content='神浜市の区新⻄区（しんせいく）ももこ‧かえで‧レナのチームが活動する地区で、やちよの⾃宅である「みかづき荘」や、みたまの「調整屋[注 16]」、うい‧灯花‧ねむが⼊院していた「⾥⾒メディカルセンター」などがある。地区内にある学校は神浜市⽴⼤附属学校で、いろはやももこたちが通っている。⽔名区（みずなく）かつての城下町で、歴史を感じさせる古い街並みが残る地区[物語 3]。'),
 Document(metadata={'source': 'data\\Wikipedia-マギアレコード_魔法少女まどか☆マギカ外伝.pdf'}, page_content='また、ひめなが所属する「神浜未来アカデミー」もある[物語 8]。⼯匠区（こうしょうく）「男⼦は職⼈として尊重されながら仕事をし、⼥は男を⽀えながら家を守る」という、やや時代錯誤な⾵習が残っている地域[物語 9]。プロミストブラッドが神浜での拠点とする旧⾞両基地がある。\x0c地区内にある学校は⼯匠学舎で、⽉咲が通っている。'),
 Document(metadata={'source': 'data\\Wikipedia-高町なのは.pdf'}, page_content='⾼町なのは出典: フリー百科事典『ウィキペディア（Wikipedia）』魔法少⼥リリカルなのはシリーズ > 登場⼈物 > ⾼町なのは⾼町 なのは（たかまち なのは）は、テレビアニメ『魔法少⼥リリカルなのは』シリーズに登場する架空の⼈物で、本編の主⼈公。『魔法戦記リリカルなのはF